In [1]:
# 필요 라이브러리 다운 
# !pip install selenium
# !pip install webdriver_manager
# !pip install html5lib

# path 지정

In [2]:
import os

# 파일이 있는 현재 위치를 path로 지정
current_directory = os.getcwd()
path = current_directory.replace('\\', '/')
path += '/' if not path.endswith('/') else ''

data_path = path + "data/"
print("Data_path :", data_path)

img_path = path + "img/"
print("IMG_path :", img_path)

#필요시 path 재조정
# path = 'C:/Users/sook7/미래에솦_주피터/본선코드정리/'
# data_path = path + "data/"
# img_path = path + "img/"

Data_path : C:/Users/sook7/미래에솦_주피터/[빅데이터_미래에솦]소스코드/data/
IMG_path : C:/Users/sook7/미래에솦_주피터/[빅데이터_미래에솦]소스코드/img/


In [3]:
#패키지 호출

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import requests

import datetime
from urllib.request import urlopen
from urllib.parse import quote_plus
from pandas.io.html import read_html
import time
import sys
import pandas as pd
import numpy as np

# 갱신이 필요한 업종 csv의 path 지정해주기

In [4]:
# CSV 파일을 읽어옴
df_ksp = pd.read_csv(data_path + 'data_KOSPI업종.csv')
df_kdq = pd.read_csv(data_path + 'data_KOSDAQ업종.csv')

# 쉼표(,) 제거 및 float로 변환
def remove_comma_and_convert_to_float(value):
    if isinstance(value, str):
        return float(value.replace(',', ''))
    return value

# '일자' 열을 제외한 모든 열에 쉼표(,) 제거 및 float로 변환
columns_to_convert = df_ksp.columns.difference(['일자'])  # '일자' 열 제외
df_ksp[columns_to_convert] = df_ksp[columns_to_convert].applymap(remove_comma_and_convert_to_float)

columns_to_convert = df_kdq.columns.difference(['일자'])  # '일자' 열 제외
df_kdq[columns_to_convert] = df_kdq[columns_to_convert].applymap(remove_comma_and_convert_to_float)

In [5]:
#KOSPI, KOSDAQ 지수 이름 
ksp_column = ['코스피', '음식료품', '섬유의복', '종이목재', '화학', '의약품', '비금속광물', '철강금속', '기계',
       '전기전자', '의료정밀', '운수장비', '유통업', '전기가스업', '건설업', '운수창고업', '통신업', '금융업',
       '증권', '보험', '서비스업', '제조업']

kdq_column = ['코스닥', '화학', '종이·목재', '제약', '일반전기전자', '의료·정밀기기', '음식료·담배',
       '운송장비·부품', '섬유·의류', '비금속', '기계·장비', '금속', 'IT H/W', 'IT S/W & SVC',
       '통신방송서비스', '유통', '운송', '오락·문화', '금융', '건설', '제조업', 'IT부품', '반도체',
       '정보기기', '통신장비', '컴퓨터서비스', '소프트웨어', '디지털컨텐츠', '인터넷', '방송서비스', '통신서비스',
       '기타제조', '출판·매체복제', '기타서비스']

In [6]:
#selenium 크롬 드라이버 설정 
#################################################################################################
# 1.번 방법
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

#2. 번 방법, 1번 실패시 2번 사용 
# driver = webdriver.Chrome(ChromeDriverManager().install())
#################################################################################################

In [7]:
driver.implicitly_wait(1)
driver.get('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201')

# KRX 정보데이터시스템, 개별지수 시세추이 크롤링
- 인터넷 상황에 따라 time.sleep() 조정 필요 
- StaleElementError 발생하면 time.sleep 시간 늘리기

### 코스피 수집

In [8]:
#개별지수 시세추이 df
df_ksp_trend = pd.DataFrame()

# '지수' 메뉴의 태그를 찾아 클릭
지수_element = driver.find_element(By.XPATH, '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/a')
지수_element.click()

time.sleep(1)

주가지수_element = driver.find_element(By.XPATH, '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/ul/li[1]/a')
주가지수_element.click()

time.sleep(1)

개별지수시세추이_element = driver.find_element(By.XPATH, '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/ul/li[1]/ul/li[3]/a')
개별지수시세추이_element.click()

time.sleep(1)

#1개월 단위로 보기 위해 1개월 버튼 클릭 
button = driver.find_element(By.XPATH, '//button[contains(text(), "6개월")]')
button.click()

time.sleep(1)

# 지수명 선택박스 클릭
element = driver.find_element(By.ID, "btnindIdx_finder_equidx0_0")
element.click()

time.sleep(1)

# 시장구분 체크박스 클릭 (KOSPI)
# KOSPI : 'label[for="mktsel_2"]'  KOSDAQ : 'label[for="mktsel_3"]
market_check_box = driver.find_element(By.CSS_SELECTOR, 'label[for="mktsel_2"]')
market_check_box.click()
time.sleep(1)

## KOSPI 종목들 순회
for idx, col in enumerate(ksp_column):
    # 지수명 선택 (ksp_column에 해당하는 지수명을 찾는다)
    element = driver.find_element(By.XPATH, f'//td[text()="{col}"]')
    #지수 번갈아가며 클릭
    element.click()
    
    #검색버튼 클릭
    search_button = driver.find_element(By.ID, "jsSearchButton")
    search_button.click()
    
    time.sleep(1)
    
    # 페이지의 HTML을 가져옴
    page_source = driver.page_source

    # BeautifulSoup을 사용하여 HTML을 파싱
    soup = BeautifulSoup(page_source, 'html.parser')

    table = soup.find('table', {'class': 'CI-GRID-BODY-TABLE'})

    # 테이블을 DataFrame으로 변환
    df = pd.read_html(str(table))[0]
    df = df.iloc[:,:2]
    df.columns = ['일자', col]
    
    # DataFrame을 기존 데이터프레임에 추가 또는 병합
    if idx == 0:
        df_ksp_trend = df
    else:
        df_ksp_trend = pd.merge(df_ksp_trend, df, on='일자', how='outer')
        
    # 지수명 선택박스 클릭 후 코스피 선택 
    element = driver.find_element(By.ID, "btnindIdx_finder_equidx0_0")
    element.click()
    time.sleep(0.5)
    market_check_box = driver.find_element(By.CSS_SELECTOR, 'label[for="mktsel_2"]')
    market_check_box.click()
    time.sleep(1)

driver.quit()

### 코스닥 수집

In [9]:
#selenium 크롬 드라이버 설정 
#################################################################################################
# 1.번 방법
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

#2. 번 방법, 1번 실패시 2번 사용 
# driver = webdriver.Chrome(ChromeDriverManager().install())
#################################################################################################

driver.implicitly_wait(1)
driver.get('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201')

df_kdq_trend = pd.DataFrame()

# '지수' 메뉴의 태그를 찾아 클릭
지수_element = driver.find_element(By.XPATH, '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/a')
지수_element.click()

time.sleep(1)

주가지수_element = driver.find_element(By.XPATH, '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/ul/li[1]/a')
주가지수_element.click()

time.sleep(1)

개별지수시세추이_element = driver.find_element(By.XPATH, '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/ul/li[1]/ul/li[3]/a')
개별지수시세추이_element.click()

time.sleep(1)

#1개월 단위로 보기 위해 1개월 버튼 클릭 
button = driver.find_element(By.XPATH, '//button[contains(text(), "6개월")]')
button.click()

time.sleep(1)

# 지수명 선택박스 클릭
element = driver.find_element(By.ID, "btnindIdx_finder_equidx0_0")
element.click()

time.sleep(1)

# 시장구분 체크박스 클릭 (KOSPI)
# KOSPI : 'label[for="mktsel_2"]'  KOSDAQ : 'label[for="mktsel_3"]
market_check_box = driver.find_element(By.CSS_SELECTOR, 'label[for="mktsel_3"]')
market_check_box.click()
time.sleep(1)

## KOSDAQ 종목들 순회
for idx, col in enumerate(kdq_column):
    # 지수명 선택
    element = driver.find_element(By.XPATH, f'//td[text()="{col}"]')

    #지수 번갈아가며 클릭
    element.click()
    
    #검색버튼 클릭
    search_button = driver.find_element(By.ID, "jsSearchButton")
    search_button.click()
    
    time.sleep(1)
    
    # 페이지의 HTML을 가져옴
    page_source = driver.page_source

    # BeautifulSoup을 사용하여 HTML을 파싱
    soup = BeautifulSoup(page_source, 'html.parser')

    table = soup.find('table', {'class': 'CI-GRID-BODY-TABLE'})

    # 테이블을 DataFrame으로 변환
    df = pd.read_html(str(table))[0]
    df = df.iloc[:,:2]
    df.columns = ['일자', col]
    
    # DataFrame을 기존 데이터프레임에 추가 또는 병합
    if idx == 0:
        df_kdq_trend = df
    else:
        df_kdq_trend = pd.merge(df_kdq_trend, df, on='일자', how='outer')
        
    # 지수명 선택박스 클릭
    element = driver.find_element(By.ID, "btnindIdx_finder_equidx0_0")
    element.click()
    time.sleep(0.5)
    market_check_box = driver.find_element(By.CSS_SELECTOR, 'label[for="mktsel_3"]')
    market_check_box.click()
    time.sleep(1)

driver.quit()

In [10]:
#새로 가져온 코스피 산업지수
df_ksp_trend

,일자,코스피,음식료품,섬유의복,종이목재,화학,의약품,비금속광물,철강금속,기계,...,유통업,전기가스업,건설업,운수창고업,통신업,금융업,증권,보험,서비스업,제조업
0,2023/09/13,2534.70,3568.75,335.60,539.73,5011.00,12601.17,3970.05,6372.26,1154.62,...,374.32,596.78,71.64,1457.98,363.57,369.67,1799.11,16929.97,1112.93,6820.86
1,2023/09/12,2536.58,3570.36,338.10,539.90,5096.13,12681.92,3993.92,6472.77,1154.30,...,377.71,595.27,71.13,1472.52,358.96,366.13,1779.37,16606.08,1122.31,6830.93
2,2023/09/11,2556.88,3561.35,343.56,536.64,5140.48,12880.27,3994.35,6672.83,1172.51,...,382.00,606.36,72.66,1483.00,360.86,368.68,1784.09,16738.99,1119.61,6892.69
3,2023/09/08,2547.68,3579.56,333.56,536.25,5187.01,12632.05,3982.87,6782.83,1158.20,...,381.59,611.59,71.59,1490.43,365.57,363.81,1787.25,16275.86,1115.56,6875.60
4,2023/09/07,2548.26,3551.20,329.67,542.46,5199.90,12511.98,3951.43,6643.17,1160.42,...,377.17,592.41,71.08,1479.30,361.87,360.74,1766.30,15907.35,1117.50,6896.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2023/03/17,2395.69,3601.89,384.71,340.62,5371.72,13155.64,3301.28,4762.52,950.00,...,322.71,753.16,74.78,1630.30,351.92,347.32,1665.72,14670.94,1137.59,6358.51
124,2023/03/16,2377.91,3569.14,383.34,337.90,5448.95,12817.08,3448.62,4782.09,946.42,...,319.63,730.43,74.49,1608.10,353.29,344.55,1650.97,14592.07,1122.03,6319.48
125,2023/03/15,2379.72,3589.26,390.13,340.06,5472.14,12743.59,3196.03,4838.01,957.10,...,322.70,730.86,75.61,1614.13,356.22,348.89,1661.01,14982.99,1121.91,6306.15
126,2023/03/14,2348.97,3528.20,391.14,335.74,5419.49,12659.51,3152.83,4704.44,927.36,...,321.28,726.68,73.26,1612.05,349.87,345.73,1657.51,14871.62,1118.38,6209.35


In [11]:
#새로 가져온 코스닥 산업지수
df_kdq_trend

,일자,코스닥,화학,종이·목재,제약,일반전기전자,의료·정밀기기,음식료·담배,운송장비·부품,섬유·의류,...,통신장비,컴퓨터서비스,소프트웨어,디지털컨텐츠,인터넷,방송서비스,통신서비스,기타제조,출판·매체복제,기타서비스
0,2023/09/13,882.72,2957.20,11588.66,8261.96,2177.51,2448.17,6827.56,5692.84,87.28,...,118.15,209.14,273.98,877.84,9793.52,2335.34,624.66,1109.35,3287.86,2810.24
1,2023/09/12,898.04,3002.56,11529.28,8431.08,2210.51,2493.36,6850.52,5681.14,88.56,...,116.22,210.61,277.89,881.71,9872.02,2368.26,624.10,1126.02,3295.97,2915.37
2,2023/09/11,912.55,3034.64,11096.35,8453.44,2277.72,2543.98,6934.56,5638.37,89.57,...,115.95,215.80,291.45,876.72,9976.49,2331.87,628.29,1131.24,3393.38,2963.05
3,2023/09/08,914.18,3063.39,11217.28,8267.69,2328.66,2534.97,6954.05,5621.95,88.17,...,115.48,216.01,289.68,878.63,9936.17,2359.59,630.05,1132.33,3316.13,2883.26
4,2023/09/07,906.36,3044.74,11490.32,8036.39,2283.48,2496.27,6847.01,5643.40,87.48,...,115.05,216.57,290.21,874.89,9919.58,2349.35,629.69,1115.82,3267.59,2822.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2023/03/17,797.39,2964.24,10890.89,7859.08,1783.60,1916.81,7881.68,5559.75,96.54,...,138.19,234.42,260.24,1079.43,10157.73,2989.12,788.64,1260.11,3835.10,2160.20
124,2023/03/16,781.98,2986.40,10966.45,7478.06,1864.59,1885.94,7705.00,5522.37,94.09,...,132.32,225.01,249.92,1028.80,9962.65,2949.14,768.88,1228.63,3751.50,2078.94
125,2023/03/15,781.17,3005.18,11084.49,7556.98,1851.96,1893.46,7813.78,5540.93,95.35,...,134.08,227.93,249.38,1043.53,10109.47,2943.36,770.05,1245.04,3790.59,2109.62
126,2023/03/14,758.05,2908.00,10971.11,7384.53,1736.09,1829.44,7552.43,5322.10,94.48,...,131.59,224.20,243.91,1038.74,9888.57,3004.31,745.97,1220.99,3799.58,2077.62


In [12]:
# ksp 업종 갱신
df_ksp_new = pd.concat([df_ksp_trend, df_ksp], ignore_index=True)
df_ksp_new.drop_duplicates(subset=['일자'], inplace=True)
df_ksp_new['일자'] = pd.to_datetime(df_ksp_new['일자'])

# kdq 업종 갱신
df_kdq_new = pd.concat([df_kdq_trend, df_kdq], ignore_index=True)
df_kdq_new.drop_duplicates(subset=['일자'], inplace=True)
df_kdq_new['일자'] = pd.to_datetime(df_kdq_new['일자'])

#reset_index
df_ksp_new.reset_index(drop=True, inplace=True)
df_kdq_new.reset_index(drop=True, inplace=True)

# 최종 저장 
# df_ksp_new.to_csv(data_path + 'data_KOSPI업종_update.csv', index=False)
# df_kdq_new.to_csv(data_path + 'data_KOSDAQ업종_update.csv',index=False)

# 업황 생성 후 KOSPI업황, KOSDAQ업황 갱신

In [13]:
data_kpi = df_ksp_new

# 1. (산업 주가지수별 해당 일자의 주가지수 - 산업 주가지수별 해당 일자의 5일전 주가지수) / 산업 주가지수별 해당 일자의 5일전 주가지수 * 100 

data_kpi_rate = data_kpi.copy()
columns = list(data_kpi_rate.columns)
columns = columns[1:] # 일자 칼럼 제외

# 5일 이전 대비 당일의 등락률
for column in columns:
    for i in range(0, data_kpi.shape[0] - 5):       
        data_kpi_rate.loc[i, column] = (data_kpi.loc[i, column] - data_kpi_rate.loc[i+4, column]) / data_kpi_rate.loc[i+4, column] * 100


# 2. 1.에서 구한 값 - 시장(KOSPI, KOSDAQ)지수 
columns = columns[1:] # 코스피 칼럼 제외

for column in columns:
    data_kpi_rate[column] = data_kpi_rate[column] - data_kpi_rate['코스피']
    
# 마지막 5개 행 삭제 
data_kpi_rate = data_kpi_rate.iloc[:-5, :]

#업데이트 필요 시 실행
# data_kpi_rate.to_csv(data_path + 'data_KOSPI업황_update.csv', index = False)

In [14]:
data_kdq = df_kdq_new

# 1. (산업 주가지수별 해당 일자의 주가지수 - 산업 주가지수별 해당 일자의 5일전 주가지수) / 산업 주가지수별 해당 일자의 5일전 주가지수 * 100 
data_kdq_rate = data_kdq.copy()
columns = list(data_kdq_rate.columns)
columns = columns[1:] # 일자 칼럼 제외

# 5일 이전 대비 당일의 등락률
for column in columns:
    for i in range(0, data_kdq.shape[0] - 5):
        data_kdq_rate.loc[i, column] = (data_kdq_rate.loc[i, column] - data_kdq_rate.loc[i+4, column]) / data_kdq_rate.loc[i+4, column] * 100

# 2. 1.에서 구한 값 - 시장(KOSPI, KOSDAQ)지수 
columns = columns[1:] # 코스닥 칼럼 제외

for column in columns:
    data_kdq_rate[column] = data_kdq_rate[column] - data_kdq_rate['코스닥']
    
#마지막 5개 행 삭제 
data_kdq_rate = data_kdq_rate.iloc[:-5,:] 

#업데이트 필요 시 실행
# data_kdq_rate.to_csv(data_path+'data_KOSDAQ업황_update.csv',index = False)

In [15]:
# 업황 제대로 계산되었는지 확인
data_kdq_rate

,일자,코스닥,화학,종이·목재,제약,일반전기전자,의료·정밀기기,음식료·담배,운송장비·부품,섬유·의류,...,통신장비,컴퓨터서비스,소프트웨어,디지털컨텐츠,인터넷,방송서비스,통신서비스,기타제조,출판·매체복제,기타서비스
0,2023-09-13,-2.608235,-0.266887,3.464086,5.415092,-2.032489,0.681360,2.324170,3.484303,2.379611,...,5.302716,-0.822526,-2.984267,2.945420,1.337415,2.011900,1.809429,2.028393,3.228570,2.162934
1,2023-09-12,-2.168963,-1.048878,-1.055305,5.608913,-4.132405,0.381849,0.904808,1.006464,3.184134,...,1.298953,-2.489253,-3.094733,2.828834,0.036444,4.373808,0.741159,1.473141,-0.476700,4.156328
2,2023-09-11,-0.969093,-1.985468,-0.442692,6.705161,-4.059823,3.147224,-0.873023,-1.714106,3.043167,...,0.046253,0.645768,0.773901,-0.007899,0.761835,0.597815,-0.375022,-0.949632,1.259024,5.703814
3,2023-09-08,-0.541799,-1.396145,16.660186,3.637787,-2.218394,3.098256,-1.997296,-2.109429,0.848966,...,-1.701492,-0.220823,5.323684,-0.735729,-0.192590,0.131133,-0.013783,-0.129265,-0.391767,2.566540
4,2023-09-07,-1.454759,-1.263919,27.889712,1.938774,-3.042104,1.649427,-2.830788,-1.524260,0.976943,...,-1.701807,1.658340,7.409487,0.047245,-0.188230,1.688133,-0.194597,-1.132848,2.133532,3.912755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3626,2009-01-15,-4.220598,-2.262396,0.695644,-1.181765,0.443550,1.514252,-0.949142,-1.017903,-2.116609,...,-0.528370,1.714525,4.649014,-1.398549,-4.214958,3.452532,-2.453486,4.415397,3.709591,-3.746769
3627,2009-01-14,2.274767,-1.507310,0.678949,-3.459390,0.264267,1.133472,0.441080,-0.382709,-0.824062,...,-1.835028,-0.711075,3.169922,-1.229457,-2.080887,0.670415,-2.109580,-1.106183,1.240983,-3.178188
3628,2009-01-13,-0.280796,-1.152355,1.657835,-2.136834,0.969294,-0.590324,1.624749,-1.141087,0.231298,...,0.100350,-0.319612,1.665665,-0.690726,-0.878127,-0.903528,3.683699,0.722611,1.737593,-2.221296
3629,2009-01-12,0.843833,-1.039677,2.712186,-1.150643,0.585685,0.325284,-0.300758,0.472131,-0.719264,...,-1.213999,-3.949701,0.463097,-0.792847,0.854987,-1.885290,5.573595,-1.658505,8.147217,-3.413227


# 공모주 시장동향 분기별 자동화

In [16]:
df_market_trend = pd.read_csv(data_path + 'data_streamlit_df.csv')
df_market_trend = df_market_trend[['기업명','종목코드','신규상장일','시초/공모(%)']]

df_market_trend['시초/공모(%)'] = pd.to_numeric(df_market_trend['시초/공모(%)'].str.replace('%', ''), errors='coerce')
df_market_trend['종목코드'] = df_market_trend['종목코드'].astype(str).str.zfill(6) 
df_market_trend['신규상장일'] = pd.to_datetime(df_market_trend['신규상장일'])

df_market_trend['상장년도'] = df_market_trend['신규상장일'].dt.year 
df_market_trend['상장월'] = df_market_trend['신규상장일'].dt.month

In [17]:
df_market_trend

,기업명,종목코드,신규상장일,시초/공모(%),상장년도,상장월
0,시지트로닉스,429270,2023-08-03,5.40,2023,8
1,엠아이큐브솔루션,373170,2023-08-04,207.50,2023,8
2,파두,440110,2023-08-07,-15.16,2023,8
3,코츠테크놀로지,448710,2023-08-10,92.31,2023,8
4,큐리옥스바이오시스템즈,445680,2023-08-10,0.00,2023,8
...,...,...,...,...,...,...
758,대한과학,131220,2011-10-11,67.00,2011,10
759,넥솔론(유가),110570,2011-10-14,50.00,2011,10
760,제닉,123330,2011-08-03,61.82,2011,8
761,아이씨디,040910,2011-08-05,84.71,2011,8


In [18]:
# 분기를 계산하는 함수 정의
def cal_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    elif month in [10, 11, 12]:
        return 4
    
# 분기를 계산하고 데이터프레임에 추가
df_market_trend['분기'] = df_market_trend['상장월'].apply(cal_quarter)

In [19]:
# 이전 분기를 계산하는 함수 정의
def cal_pre_quarter(year, quarter):
    if quarter == 1:
        return year - 1, 4
    else:
        return year, quarter - 1

# 각 행에 대해 이전 분기를 계산하고 데이터프레임에 추가
df_market_trend['이전 분기'] = df_market_trend.apply(lambda row: cal_pre_quarter(row['상장년도'], row['분기']), axis=1)

In [20]:
# 이전 분기의 '공모주 시장 동향'을 계산하는 함수 정의
def cal_pre_quarter_market_trend(row):
    previous_year, previous_quarter = row['이전 분기']
    previous_data = df_market_trend[(df_market_trend['상장년도'] == previous_year) & (df_market_trend['분기'] == previous_quarter)]
    if len(previous_data) > 0:
        previous_positive_percentage = previous_data[previous_data['시초/공모(%)'] > 0]
        return len(previous_positive_percentage) / len(previous_data)
    else:
        return np.nan

# 각 행에 대해 이전 분기의 '공모주 시장 동향'을 계산하고 데이터프레임에 추가
df_market_trend['공모주 시장 동향'] = df_market_trend.apply(cal_pre_quarter_market_trend, axis=1)

# 결과 확인
df_market_trend = df_market_trend[['기업명', '종목코드', '상장년도', '상장월', '분기', '이전 분기', '공모주 시장 동향']]
df_market_trend.head(5)

,기업명,종목코드,상장년도,상장월,분기,이전 분기,공모주 시장 동향
0,시지트로닉스,429270,2023,8,3,"(2023, 2)",0.846154
1,엠아이큐브솔루션,373170,2023,8,3,"(2023, 2)",0.846154
2,파두,440110,2023,8,3,"(2023, 2)",0.846154
3,코츠테크놀로지,448710,2023,8,3,"(2023, 2)",0.846154
4,큐리옥스바이오시스템즈,445680,2023,8,3,"(2023, 2)",0.846154


In [21]:
# 분기별 공모주 시장동향 csv 저장 
df_market_trend = pd.DataFrame(df_market_trend.groupby('이전 분기')['공모주 시장 동향'].unique())
df_market_trend.reset_index(inplace=True)
df_market_trend['공모주 시장 동향'] = round(df_market_trend['공모주 시장 동향'].astype(float), 4)
# df_market_trend.to_csv(data_path+'data_공모주시장동향(분기).csv')
df_market_trend

,이전 분기,공모주 시장 동향
0,"(2010, 4)",NaN
1,"(2011, 2)",NaN
2,"(2011, 3)",1.0000
3,"(2011, 4)",0.6667
4,"(2012, 1)",1.0000
5,"(2012, 2)",0.5000
6,"(2012, 3)",0.6250
7,"(2012, 4)",0.5556
8,"(2013, 1)",0.7500
9,"(2013, 2)",1.0000
